In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
import glob
import os
import dask.dataframe as dd

/opt/conda/lib/python3.11/site-packages/dask/dataframe/_pyarrow_compat.py:15: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 13.0.0. Please consider upgrading.
  warnings.warn(


In [2]:
filelist = np.sort(glob.glob('/arc/projects/k-pop/catalogues/skymapperdr4/*'))
print(filelist)

['/arc/projects/k-pop/catalogues/skymapperdr4/LISTFILE'
 '/arc/projects/k-pop/catalogues/skymapperdr4/SkyMapper.DR4.master.part001.csv.gz'
 '/arc/projects/k-pop/catalogues/skymapperdr4/SkyMapper.DR4.master.part002.csv.gz'
 '/arc/projects/k-pop/catalogues/skymapperdr4/SkyMapper.DR4.master.part003.csv.gz'
 '/arc/projects/k-pop/catalogues/skymapperdr4/SkyMapper.DR4.master.part004.csv.gz'
 '/arc/projects/k-pop/catalogues/skymapperdr4/SkyMapper.DR4.master.part005.csv.gz'
 '/arc/projects/k-pop/catalogues/skymapperdr4/SkyMapper.DR4.master.part006.csv.gz'
 '/arc/projects/k-pop/catalogues/skymapperdr4/SkyMapper.DR4.master.part007.csv.gz'
 '/arc/projects/k-pop/catalogues/skymapperdr4/SkyMapper.DR4.master.part008.csv.gz'
 '/arc/projects/k-pop/catalogues/skymapperdr4/SkyMapper.DR4.master.part009.csv.gz'
 '/arc/projects/k-pop/catalogues/skymapperdr4/SkyMapper.DR4.master.part010.csv.gz'
 '/arc/projects/k-pop/catalogues/skymapperdr4/SkyMapper.DR4.master.part011.csv.gz'
 '/arc/projects/k-pop/catalogue

## Just taking the first file as an example and will automate later

In [3]:
file_to_read = filelist[1]

In [4]:
usecols = ['flags','nimaflags','class_star','u_psf','e_u_psf','v_psf','e_v_psf',
           'g_psf','e_g_psf','r_psf','e_r_psf','i_psf','e_i_psf','z_psf','e_z_psf',
           'gaia_dr3_id1','gaia_dr3_dist1']
keepcols = ['u_psf','e_u_psf','v_psf','e_v_psf',
           'g_psf','e_g_psf','r_psf','e_r_psf','i_psf','e_i_psf','z_psf','e_z_psf',
           'gaia_dr3_id1']
dtype_dict = {}
for col in usecols:
    if (col == 'gaia_dr3_id1') | (col == 'flags') | (col == 'nimaflags'):
        dtype_dict[col] = 'Int64'
    else:
        dtype_dict[col] = 'float64'

In [32]:
df = dd.read_csv(file_to_read,
                 compression='gzip',
                 usecols=['flags','nimaflags','class_star','u_psf','e_u_psf','v_psf','e_v_psf',
                          'g_psf','e_g_psf','r_psf','e_r_psf','i_psf','e_i_psf','z_psf','e_z_psf',
                          'gaia_dr3_id1','gaia_dr3_dist1'],
                 dtype=dtype_dict)

In [33]:
mask = ((df['class_star'] > 0.9) & (df['r_psf'] < 19)) | ((df['class_star'] > 0.5) & (df['r_psf'] >= 19)) & (df['flags'] < 4) & (df['nimaflags'] < 5) & (df['gaia_dr3_dist1'] < 2)
df = df[mask]
df = df[usecols]

In [35]:
result = df.compute()

In [36]:
result

,flags,nimaflags,class_star,u_psf,e_u_psf,v_psf,e_v_psf,g_psf,e_g_psf,r_psf,e_r_psf,i_psf,e_i_psf,z_psf,e_z_psf,gaia_dr3_id1,gaia_dr3_dist1
0,0,0,0.976,NaN,NaN,NaN,NaN,20.0442,0.0196,19.2577,0.0227,18.7571,0.0246,18.5266,0.0665,6780195833444243584,0.09
1,3,3,0.983,NaN,NaN,19.0020,0.0419,18.1890,0.0105,17.8684,0.0228,17.6961,0.0136,17.5714,0.0137,6780196043899539840,0.05
2,0,0,0.955,NaN,NaN,NaN,NaN,NaN,NaN,20.0497,0.0334,19.3303,0.0439,19.0356,0.0664,6780195867803981440,0.12
3,0,0,0.984,NaN,NaN,18.9084,0.0906,17.8585,0.0128,17.5662,0.0123,17.3876,0.0120,17.3198,0.0121,6780196078259277568,0.10
4,0,0,0.984,NaN,NaN,NaN,NaN,17.8258,0.0218,17.2761,0.0211,16.9385,0.0072,16.8106,0.0088,6780196078259276672,0.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999994,0,0,0.983,NaN,NaN,18.8927,0.1144,16.8566,0.0141,16.2730,0.0151,15.9202,0.0059,15.7705,0.0078,6814248292672438656,0.33
999995,0,0,0.877,NaN,NaN,NaN,NaN,NaN,NaN,19.7201,0.0414,18.7638,0.0222,18.5008,0.0348,6814248292672440704,0.19
999996,0,0,0.991,17.059,0.0208,16.6893,0.0240,15.2526,0.0078,14.8521,0.0110,14.6083,0.0054,14.5209,0.0077,6814248228247960320,0.12
999997,0,0,0.531,NaN,NaN,NaN,NaN,20.0349,0.0641,19.8884,0.0581,19.4370,0.0674,NaN,NaN,6814236472922433408,0.36


In [34]:
df.gaia_dr3_id1.value_counts().compute()

/opt/conda/lib/python3.11/site-packages/dask/dataframe/io/csv.py:555: UserWarning: Warning gzip compression does not support breaking apart files
Please ensure that each individual file can fit in memory and
use the keyword ``blocksize=None to remove this message``
Setting ``blocksize=None``
  warn(


gaia_dr3_id1
6569860014965275520    1
6569860667800320384    1
6569860775176683520    1
6569860809536421248    1
6569861015694852352    1
                      ..
6833460677820238720    1
6833460716474995584    1
6833460716474996224    1
6833460716474996608    1
6833461742972146560    1
Name: count, Length: 717652, dtype: Int64

In [39]:
# Step 1: compute the minimum distance per crossmatch ID
min_dist = df.groupby("gaia_dr3_id1")["gaia_dr3_dist1"].min().reset_index()

# Step 2: merge this back to the filtered dataframe to keep only the closest matches
result = df.merge(min_dist, on=["gaia_dr3_id1", "gaia_dr3_dist1"])

In [40]:
result.compute()

,flags,nimaflags,class_star,u_psf,e_u_psf,v_psf,e_v_psf,g_psf,e_g_psf,r_psf,e_r_psf,i_psf,e_i_psf,z_psf,e_z_psf,gaia_dr3_id1,gaia_dr3_dist1
0,0,0,0.976,NaN,NaN,NaN,NaN,20.0442,0.0196,19.2577,0.0227,18.7571,0.0246,18.5266,0.0665,6780195833444243584,0.09
1,3,3,0.983,NaN,NaN,19.0020,0.0419,18.1890,0.0105,17.8684,0.0228,17.6961,0.0136,17.5714,0.0137,6780196043899539840,0.05
2,0,0,0.955,NaN,NaN,NaN,NaN,NaN,NaN,20.0497,0.0334,19.3303,0.0439,19.0356,0.0664,6780195867803981440,0.12
3,0,0,0.984,NaN,NaN,18.9084,0.0906,17.8585,0.0128,17.5662,0.0123,17.3876,0.0120,17.3198,0.0121,6780196078259277568,0.10
4,0,0,0.984,NaN,NaN,NaN,NaN,17.8258,0.0218,17.2761,0.0211,16.9385,0.0072,16.8106,0.0088,6780196078259276672,0.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717647,0,0,0.983,NaN,NaN,18.8927,0.1144,16.8566,0.0141,16.2730,0.0151,15.9202,0.0059,15.7705,0.0078,6814248292672438656,0.33
717648,0,0,0.877,NaN,NaN,NaN,NaN,NaN,NaN,19.7201,0.0414,18.7638,0.0222,18.5008,0.0348,6814248292672440704,0.19
717649,0,0,0.991,17.059,0.0208,16.6893,0.0240,15.2526,0.0078,14.8521,0.0110,14.6083,0.0054,14.5209,0.0077,6814248228247960320,0.12
717650,0,0,0.531,NaN,NaN,NaN,NaN,20.0349,0.0641,19.8884,0.0581,19.4370,0.0674,NaN,NaN,6814236472922433408,0.36


## Now doing it for all files

In [5]:
df = dd.read_csv('/arc/projects/k-pop/catalogues/skymapperdr4/*.csv.gz',
                 compression='gzip',
                 usecols=['flags','nimaflags','class_star','u_psf','e_u_psf','v_psf','e_v_psf',
                          'g_psf','e_g_psf','r_psf','e_r_psf','i_psf','e_i_psf','z_psf','e_z_psf',
                          'gaia_dr3_id1','gaia_dr3_dist1'],
                 dtype=dtype_dict)

/opt/conda/lib/python3.11/site-packages/dask/dataframe/io/csv.py:555: UserWarning: Warning gzip compression does not support breaking apart files
Please ensure that each individual file can fit in memory and
use the keyword ``blocksize=None to remove this message``
Setting ``blocksize=None``
  warn(


In [6]:
mask = ((df['class_star'] > 0.9) & (df['r_psf'] < 19)) | ((df['class_star'] > 0.5) & (df['r_psf'] >= 19)) & (df['flags'] < 4) & (df['nimaflags'] < 5) & (df['gaia_dr3_dist1'] < 2)
df = df[mask]

# Step 1: compute the minimum distance per crossmatch ID
min_dist = df.groupby("gaia_dr3_id1")["gaia_dr3_dist1"].min().reset_index()

# Step 2: merge this back to the filtered dataframe to keep only the closest matches
df = df.merge(min_dist, on=["gaia_dr3_id1", "gaia_dr3_dist1"])

In [7]:
df = df[keepcols]

In [8]:
df

,u_psf,e_u_psf,v_psf,e_v_psf,g_psf,e_g_psf,r_psf,e_r_psf,i_psf,e_i_psf,z_psf,e_z_psf,gaia_dr3_id1
npartitions=725,,,,,,,,,,,,,
,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,Int64
,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...


In [9]:
df['source_id'] = df['gaia_dr3_id1']
df = df.drop('gaia_dr3_id1', axis=1)

In [10]:
new_order = ['source_id','u_psf','e_u_psf','v_psf','e_v_psf','g_psf','e_g_psf','r_psf','e_r_psf','i_psf','e_i_psf','z_psf','e_z_psf']
df = df[new_order]
df = df.dropna(subset=['source_id'])
df['source_id'] = df['source_id'].astype('int64')
df

,source_id,u_psf,e_u_psf,v_psf,e_v_psf,g_psf,e_g_psf,r_psf,e_r_psf,i_psf,e_i_psf,z_psf,e_z_psf
npartitions=725,,,,,,,,,,,,,
,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...


In [12]:
df.to_hdf('/arc/projects/k-pop/catalogues/adql_matches/smssdr4_curated.h5',
          key='catalog', mode='w', format='table', complib='blosc', complevel=9)

['/arc/projects/k-pop/catalogues/adql_matches/smssdr4_curated.h5',
 '/arc/projects/k-pop/catalogues/adql_matches/smssdr4_curated.h5',
 '/arc/projects/k-pop/catalogues/adql_matches/smssdr4_curated.h5',
 '/arc/projects/k-pop/catalogues/adql_matches/smssdr4_curated.h5',
 '/arc/projects/k-pop/catalogues/adql_matches/smssdr4_curated.h5',
 '/arc/projects/k-pop/catalogues/adql_matches/smssdr4_curated.h5',
 '/arc/projects/k-pop/catalogues/adql_matches/smssdr4_curated.h5',
 '/arc/projects/k-pop/catalogues/adql_matches/smssdr4_curated.h5',
 '/arc/projects/k-pop/catalogues/adql_matches/smssdr4_curated.h5',
 '/arc/projects/k-pop/catalogues/adql_matches/smssdr4_curated.h5',
 '/arc/projects/k-pop/catalogues/adql_matches/smssdr4_curated.h5',
 '/arc/projects/k-pop/catalogues/adql_matches/smssdr4_curated.h5',
 '/arc/projects/k-pop/catalogues/adql_matches/smssdr4_curated.h5',
 '/arc/projects/k-pop/catalogues/adql_matches/smssdr4_curated.h5',
 '/arc/projects/k-pop/catalogues/adql_matches/smssdr4_curated.

In [13]:
df

,source_id,u_psf,e_u_psf,v_psf,e_v_psf,g_psf,e_g_psf,r_psf,e_r_psf,i_psf,e_i_psf,z_psf,e_z_psf
npartitions=725,,,,,,,,,,,,,
,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...


In [14]:
import h5py

In [16]:
with h5py.File('/arc/projects/k-pop/catalogues/adql_matches/smssdr4_curated.h5', 'r') as f:
    print(list(f.keys()))  # List top-level keys
    print(list(f['catalog'].keys()))  # List keys under the dataset_name group
    data = f['catalog']['table'][:50]

['catalog']
['_i_table', 'table']


In [17]:
data

array([( 0, [6780195833444243584], [        nan,         nan,         nan,         nan, 2.00442e+01, 1.96000e-02, 1.92577e+01, 2.27000e-02, 1.87571e+01, 2.46000e-02, 1.85266e+01, 6.65000e-02]),
       ( 1, [6780196043899539840], [        nan,         nan, 1.90020e+01, 4.19000e-02, 1.81890e+01, 1.05000e-02, 1.78684e+01, 2.28000e-02, 1.76961e+01, 1.36000e-02, 1.75714e+01, 1.37000e-02]),
       ( 2, [6780195867803981440], [        nan,         nan,         nan,         nan,         nan,         nan, 2.00497e+01, 3.34000e-02, 1.93303e+01, 4.39000e-02, 1.90356e+01, 6.64000e-02]),
       ( 3, [6780196078259277568], [        nan,         nan, 1.89084e+01, 9.06000e-02, 1.78585e+01, 1.28000e-02, 1.75662e+01, 1.23000e-02, 1.73876e+01, 1.20000e-02, 1.73198e+01, 1.21000e-02]),
       ( 4, [6780196078259276672], [        nan,         nan,         nan,         nan, 1.78258e+01, 2.18000e-02, 1.72761e+01, 2.11000e-02, 1.69385e+01, 7.20000e-03, 1.68106e+01, 8.80000e-03]),
       ( 5, [67801958678039927